In [79]:
import pandas as pd
import zipfile
import elopy
import heapq

In [26]:
with zipfile.ZipFile('data/contest_results.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_files')
df = pd.read_csv('./extracted_files/contest_results.csv', date_parser='contest_date')

/var/folders/0s/9k5wnkxx22l_g6lmt5xw673m0000gq/T/ipykernel_87261/1544680467.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('./extracted_files/contest_results.csv', date_parser='contest_date')


In [27]:
df['organization'].value_counts()

organization
npc     616391
npcw     92896
ifbb     57393
cpa      17027
Name: count, dtype: int64

In [28]:
df_pro = df[df['organization'] == 'ifbb']
df_pro_open = df_pro[df_pro['class'] == 'open']

In [29]:
df_pro = df_pro[['competitor_name', 'contest_name', 'contest_date', 'division', 'placing']]

In [67]:
df_merge = pd.merge(df_pro,
                    df_pro,
                    on=['contest_name', 'contest_date', 'division'],
                    how='inner',
                    suffixes=('_winner', '_loser'))

In [68]:
df_merge = df_merge[df_merge['placing_winner'] < df_merge['placing_loser']]

In [69]:
df_merge = df_merge[['competitor_name_winner', 'placing_winner', 'competitor_name_loser', 'placing_loser', 'contest_date', 'contest_name', 'division']]

In [70]:
df_merge.head()

,competitor_name_winner,placing_winner,competitor_name_loser,placing_loser,contest_date,contest_name,division
12,michele mayberry,11.0,paula gulman-williams,12.0,2012-02-18,ifbb flex pro,fitness
24,sheri vucick,10.0,paula gulman-williams,12.0,2012-02-18,ifbb flex pro,fitness
25,sheri vucick,10.0,michele mayberry,11.0,2012-02-18,ifbb flex pro,fitness
36,allison ethier,9.0,paula gulman-williams,12.0,2012-02-18,ifbb flex pro,fitness
37,allison ethier,9.0,michele mayberry,11.0,2012-02-18,ifbb flex pro,fitness


In [71]:
df_merge[df_merge['contest_name'].str.contains('olympia')]['contest_date'].value_counts()

contest_date
2014-06-10    5531
2022-12-13    4626
2014-06-16    2848
2023-11-03    2797
2016-12-28    2206
2021-12-30    2190
2018-09-14    2073
2020-12-18    1887
2015-09-17    1848
2017-09-15    1839
2019-09-13    1758
2014-09-18    1184
2012-09-28    1098
2013-09-26     988
2012-12-02     735
2012-12-06     390
Name: count, dtype: int64

In [72]:
olympia_2022 = '2022-12-13'
olympia_2023 = '2023-11-03'

In [73]:
df_2023 = df_merge[df_merge['contest_date'] < olympia_2023]
df_2023 = df_2023.sort_values(by='contest_date', ascending=True)

In [74]:
from elopy.elo import Elo

competitors = {}

In [75]:
for index, row in df_2023.iterrows():
    winner = row['competitor_name_winner']
    loser = row['competitor_name_loser']
    rank_diff = row['placing_loser'] - row['placing_winner']
    if winner not in competitors:
        competitors[winner] = Elo()
    if loser not in competitors:
        competitors[loser] = Elo()
    competitors[winner].play_game(competitors[loser], rank_diff)

In [81]:
bikini_olympia = df_pro[(df_pro['contest_date'] == olympia_2023)
       & (df_pro['division'] == 'bikini')]['competitor_name'].unique()

bikini_olympia = [name for name in bikini_olympia if name in competitors]

In [87]:
top_ten = []
name_rating = [(-competitors[name].elo, name) for name in bikini_olympia]
heapq.heapify(name_rating)
for i in range(15):
    print(heapq.heappop(name_rating))

(-2292.7335793102006, 'ashley kaltwasser')
(-2291.1487025403267, 'jennifer dorie')
(-2271.39177987202, 'lauralie chapados')
(-2224.7366653044833, 'elisa pecini')
(-2193.374561852178, 'maureen blanquisco')
(-2097.256944411258, 'daraja hill')
(-2080.747656703822, 'aimee leann delgado')
(-2055.120608348572, 'jourdanne lee')
(-2012.063029042556, 'romina basualdo')
(-1983.0619768109746, 'phoebe hagan')
(-1982.9256986306534, 'ivanna escandar')
(-1962.565067843334, 'ashlyn little')
(-1938.3613773760399, 'reijuana harley')
(-1933.119829179901, 'eli fernandez')
(-1919.929304882525, 'yurika shigemoto')


In [88]:
df_2022 = df_merge[df_merge['contest_date'] < olympia_2022]
df_2022 = df_2022.sort_values(by='contest_date', ascending=True)

competitors = {}

for index, row in df_2022.iterrows():
    winner = row['competitor_name_winner']
    loser = row['competitor_name_loser']
    rank_diff = row['placing_loser'] - row['placing_winner']
    if winner not in competitors:
        competitors[winner] = Elo()
    if loser not in competitors:
        competitors[loser] = Elo()
    competitors[winner].play_game(competitors[loser], rank_diff)

bikini_olympia = df_pro[(df_pro['contest_date'] == olympia_2022)
       & (df_pro['division'] == 'bikini')]['competitor_name'].unique()

bikini_olympia = [name for name in bikini_olympia if name in competitors]

(-2277.931827359975, 'janet layug')
(-2257.886982595582, 'lauralie chapados')
(-2256.2749367738516, 'ashley kaltwasser')
(-2254.1088813918072, 'elisa pecini')
(-2247.0200779276165, 'jennifer dorie')


In [89]:
top_ten = []
name_rating = [(-competitors[name].elo, name) for name in bikini_olympia]
heapq.heapify(name_rating)
for i in range(10):
    print(heapq.heappop(name_rating))

(-2277.931827359975, 'janet layug')
(-2257.886982595582, 'lauralie chapados')
(-2256.2749367738516, 'ashley kaltwasser')
(-2254.1088813918072, 'elisa pecini')
(-2247.0200779276165, 'jennifer dorie')
(-2088.727205079298, 'india paulino')
(-2086.3532917075217, 'maureen blanquisco')
(-2023.592664741882, 'jourdanne lee')
(-2006.6100386349601, 'ashlyn little')
(-1978.522322412744, 'romina basualdo')


In [91]:
competitors['ashley kaltwasser'].win_probs(competitors['jennifer dorie'])

0.6522448806741087